In [ ]:
# O primeiro passo é instalar a biblioteca de cryptography
!pip install cryptography

In [ ]:
# Aqui importamos a biblioteca do sistema operacional
import os
# E as partes da biblioteca cryptography que vamos utilizar
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding

In [ ]:
# Aqui utilizamos a função 'urandom' para gerar uma sequencia pseudo-aleatório de 32 bytes
# Ou seja, 256 bits, que será a chave simétrica do nosso algoritmo.
key = os.urandom(32)
# Aqui geramos um vetor de inicialização de bytes aleatórios exigitos para o CBC (Cipher Block Chaining)
# O vetor precisa ter o mesmo número de bytes que o tamanho de bloco utilizado no algoritmo (16 bytes nesse caso)
# Esse vetor deve mudar para cada mensagem criptografada
iv = os.urandom(16)

In [ ]:
# Escolhemos o algoritmo que será usado, neste caso o AES
algorithm = algorithms.AES(key)

# E o modo de operação do algoritmo. Aqui usamos CBC (Cipher Block Chaining) que é um modo 
# de operação para algoritmos de cifras de bloco (AES é um deles), pois é considerado criptograficamente seguro
operation_mode = modes.CBC(iv)

# Com isso criamos o objeto que será encarregado pro criptografar/descriptografar as mensagens
cipher = Cipher(algorithm, operation_mode)
encryptor = cipher.encryptor()
# E criptografamos uma mensagen qualquer
ct = encryptor.update(b"A secret message") + encryptor.finalize()

In [ ]:
# E no fim recuperamos a mensagem original
decryptor = cipher.decryptor()
print(decryptor.update(ct) + decryptor.finalize())

In [ ]:
# Se você tentar mudar a mensagem acima para uma outra qualquer provavelmente não funcionar, mas por que?
# Isso acontece  porque a mensagem precisar ter um tamanho múltiplo do tamanho do bloco usado pelo algoritmo 
# No caso do AES o bloco tem 16 bytes (ou 128 bits), como visto abaixo
print("Tamanho do bloco em bits:", algorithm.block_size)

# Para contornar esse problema precisamos usar uma função de padding
# A função de padding completa o espaço que está faltando para atingir o tamanho do bloco
# O parametro '128' é o tamanho do bloco do AES
padder = padding.PKCS7(128).padder()

original_data = padder.update(b"Agora essa mensagem pode ser de qualquer tamanho")
print("Mensagem original:", original_data)

padded_data = original_data + padder.finalize()
print("Mensagem com padding:", padded_data)

In [ ]:
# Agora fazemos todo o processo novamente para criptografar a mensagem
iv = os.urandom(16)

algorithm = algorithms.AES(key)
operation_mode = modes.CBC(iv)

cipher = Cipher(algorithm, operation_mode)
encryptor = cipher.encryptor()

ct = encryptor.update(padded_data) + encryptor.finalize()

# E no fim recuperamos a mensagem original
decryptor = cipher.decryptor()
message = decryptor.update(ct) + decryptor.finalize()

# Note como ela ainda tem o padding
print("Mesagem descriptografada:", message)

# Basta removermos o padding
unpadder = padding.PKCS7(128).unpadder()
message_no_padding = unpadder.update(message)

print("Mesagem descriptografada e sem padding:", message_no_padding)

